<a href="https://colab.research.google.com/github/AnderssonTom/D7047E-Lab-0/blob/main/D7047E_Lab_0_2_G25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#D7047E - Lab 0 - Pratical Task 0.2 - Group 25 (G25) - 2025-03-30
Group 25 (lab 0 mandatory): Tom Andersson

Group 25 (lab 0 non-mandatory): Antonino Davolos, Christos Michail, Felix Hessinger, Sandra Sandström

#Practical Task 0.2.1

In [1]:
# EXPERIMENT 1: TRAINING AlexNet ON CIFAR-10

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models

# CIFAR-10 transformations for AlexNet
transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224 for AlexNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained model
])

# Loading CIFAR-10 data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:05<00:00, 31.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
# Create an AlexNet mode
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Number of classes
num_classes = 10

# Add a fully connected layer with 10 outputs
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Function to train the model
def train_model(model, trainloader, criterion, optimizer, num_epochs=5):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Track accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%")

# Function to evaluate the model
def evaluate_model(model, testloader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Full model training, fine-tuning the AlexNet model
train_model(alexnet, trainloader, criterion, optimizer, num_epochs=5)

# Evaluate the model
evaluate_model(alexnet, testloader)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 124MB/s]


Epoch [1/5], Loss: 0.6980, Accuracy: 79.52%
Epoch [2/5], Loss: 0.1771, Accuracy: 86.91%
Epoch [3/5], Loss: 0.4929, Accuracy: 89.45%
Epoch [4/5], Loss: 0.5600, Accuracy: 90.99%
Epoch [5/5], Loss: 0.0721, Accuracy: 92.33%
Test Accuracy: 88.95%


In [4]:
# EXPERIMENT 2: APPLY PRE-TRAINED AlexNet TO CIFAR-10

# Load AlexNet pre-trained model
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Freeze except last classification layer
for param in alexnet.features.parameters():
    param.requires_grad = False

# Change Add a fully connected layer with 10 outputs
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

# Transfer model to device (GPU/CPU)
alexnet = alexnet.to(device)

# Optimizer for the last layer
optimizer = optim.SGD(alexnet.classifier[6].parameters(), lr=0.001, momentum=0.9)

# Training the model (Feature Extraction)
train_model(alexnet, trainloader, criterion, optimizer, num_epochs=5)

# Evaluate
evaluate_model(alexnet, testloader)

Epoch [1/5], Loss: 0.5152, Accuracy: 73.26%
Epoch [2/5], Loss: 0.4979, Accuracy: 77.49%
Epoch [3/5], Loss: 0.5089, Accuracy: 78.76%
Epoch [4/5], Loss: 0.3391, Accuracy: 79.31%
Epoch [5/5], Loss: 1.3469, Accuracy: 79.60%
Test Accuracy: 82.05%


##Key Differences between experiment 1 and 2

In experiment 1 (fine-tuning), we train all layers of the AlexNet, all its weights. In experiment 2(feature extraction), we only train the last output layer, making use of pre-trained features/weights for other layers.

##Explanation of difference in performance

When training the whole net, and not just the last output layer, there are more degress of freedom in training, allowing for a better fit of model and images.

#Practical Task 0.2.2

In [4]:
# TRAINING A CNN MODEL ON MNIST IMAGES

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define CNN model for MNIST
def create_cnn(input_shape=(28, 28, 1)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Train model on MNIST
mnist_model = create_cnn()
mnist_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
mnist_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

# Evaluate on MNIST
test_loss, test_acc = mnist_model.evaluate(x_test, y_test)
print(f"MNIST Test Accuracy: {test_acc:.4f}")

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 75s 39ms/step - accuracy: 0.8875 - loss: 0.3526 - val_accuracy: 0.9861 - val_loss: 0.0436
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 74s 34ms/step - accuracy: 0.9843 - loss: 0.0491 - val_accuracy: 0.9877 - val_loss: 0.0383
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 33ms/step - accuracy: 0.9898 - loss: 0.0342 - val_accuracy: 0.9883 - val_loss: 0.0329
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 32ms/step - accuracy: 0.9923 - loss: 0.0241 - val_accuracy: 0.9912 - val_loss: 0.0254
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 33ms/step - accuracy: 0.9944 - loss: 0.0183 - val_accuracy: 0.9900 - val_loss: 0.0329
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9874 - loss: 0.0395
MNIST Test Accuracy: 0.9900


In [9]:
# IMPORTING SVHN IMAGES

import requests

# URLs for SVHN dataset
train_url = "http://ufldl.stanford.edu/housenumbers/train_32x32.mat"
test_url = "http://ufldl.stanford.edu/housenumbers/test_32x32.mat"

# Function to download and save file
def download_svhn(url, filename):
    response = requests.get(url, stream=True)
    with open(filename, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    print(f"Downloaded {filename}")

# Download train and test sets
download_svhn(train_url, "train_32x32.mat")
download_svhn(test_url, "test_32x32.mat")

Downloaded train_32x32.mat
Downloaded test_32x32.mat


In [7]:
# APPLYING PRE-TRAINED MODEL/FEATURE EXTRACTION

import numpy as np
from scipy.io import loadmat

# Load the downloaded dataset
svhn_train = loadmat("train_32x32.mat")
svhn_test = loadmat("test_32x32.mat")

# Preprocess SVHN dataset
x_train_svhn = np.moveaxis(svhn_train['X'], -1, 0) / 255.0
x_test_svhn = np.moveaxis(svhn_test['X'], -1, 0) / 255.0

y_train_svhn = to_categorical(svhn_train['y'].flatten() % 10, 10)
y_test_svhn = to_categorical(svhn_test['y'].flatten() % 10, 10)

# Redefine CNN for SVHN with (32,32,3) input
svhn_model = create_cnn(input_shape=(32, 32, 3))

# Transfer weights from MNIST model (excluding input layer)
for i, layer in enumerate(mnist_model.layers[1:-2]):  # Skip first and last 2 layers
    svhn_model.layers[i+1].set_weights(layer.get_weights())
    svhn_model.layers[i+1].trainable = False  # Freeze pretrained layers

# Compile and train on SVHN
svhn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
svhn_model.fit(x_train_svhn, y_train_svhn, epochs=5, batch_size=64, validation_data=(x_test_svhn, y_test_svhn))

# Evaluate on SVHN
test_loss, test_acc = svhn_model.evaluate(x_test_svhn, y_test_svhn)
print(f"SVHN Test Accuracy: {test_acc:.4f}")

Epoch 1/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 96s 82ms/step - accuracy: 0.4426 - loss: 1.6541 - val_accuracy: 0.7487 - val_loss: 0.8907
Epoch 2/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 134s 75ms/step - accuracy: 0.7685 - loss: 0.7905 - val_accuracy: 0.7750 - val_loss: 0.8034
Epoch 3/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 142s 75ms/step - accuracy: 0.7963 - loss: 0.6941 - val_accuracy: 0.7853 - val_loss: 0.7652
Epoch 4/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 90s 79ms/step - accuracy: 0.8147 - loss: 0.6336 - val_accuracy: 0.7986 - val_loss: 0.7163
Epoch 5/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 146s 82ms/step - accuracy: 0.8264 - loss: 0.5915 - val_accuracy: 0.7857 - val_loss: 0.7481
814/814 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.7849 - loss: 0.7576
SVHN Test Accuracy: 0.7857


In [8]:
# TRANSFER LEARNING

# Unfreeze all layers
for layer in svhn_model.layers:
    layer.trainable = True  # Allow all weights to be updated

# Recompile
svhn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train entire model
svhn_model.fit(x_train_svhn, y_train_svhn, epochs=5, batch_size=64, validation_data=(x_test_svhn, y_test_svhn))

# Evaluate
test_loss, test_acc = svhn_model.evaluate(x_test_svhn, y_test_svhn)
print(f"SVHN Test Accuracy: {test_acc:.4f}")

Epoch 1/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 125s 106ms/step - accuracy: 0.8306 - loss: 0.5745 - val_accuracy: 0.8306 - val_loss: 0.5934
Epoch 2/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 124s 91ms/step - accuracy: 0.8641 - loss: 0.4540 - val_accuracy: 0.8532 - val_loss: 0.5203
Epoch 3/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 102s 89ms/step - accuracy: 0.8813 - loss: 0.3964 - val_accuracy: 0.8583 - val_loss: 0.4880
Epoch 4/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 102s 89ms/step - accuracy: 0.8941 - loss: 0.3513 - val_accuracy: 0.8660 - val_loss: 0.4658
Epoch 5/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 162s 107ms/step - accuracy: 0.9027 - loss: 0.3254 - val_accuracy: 0.8691 - val_loss: 0.4644
814/814 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.8665 - loss: 0.4776
SVHN Test Accuracy: 0.8691
